In [1]:
source_root = "path_to_folder_with_xml_files_in_it"
out_path = "./out.sqlite"

In [2]:
source_root = "/Users/jan/Wallpapers/3dprinting.stackexchange.com"

In [3]:
# We will store posts in a sqlite database
import sqlite3
from sqlite3 import Error
# We use xml.etree to parse xml tags
import xml.etree.ElementTree as ET
from tqdm import tqdm
import os
from collections import OrderedDict

# More info here:
# https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

posts_schema = OrderedDict()
posts_schema["id"] = ("Id", "INTEGER PRIMARY KEY")
posts_schema["post_type"] = ("PostTypeId", "INTEGER NOT NULL")
posts_schema["accepted_answer_id"] = ("AcceptedAnswerId", "INTEGER")
posts_schema["creation_date"] = ("CreationDate", "TEXT NOT NULL")
posts_schema["score"] = ("Score", "INTEGER NOT NULL")
posts_schema["view_count"] = ("ViewCount", "INTEGER")
posts_schema["body"] = ("Body", "TEXT")
posts_schema["user_id"] = ("OwnerUserId", "INTEGER")
posts_schema["last_activity_date"] = ("LastActivityDate", "TEXT")
posts_schema["title"] = ("Title", "TEXT")
posts_schema["tags"] = ("Tags", "TEXT")
posts_schema["answer_count"] = ("AnswerCount", "INTEGER")
posts_schema["comment_count"] = ("CommentCount", "INTEGER")


users_schema = OrderedDict()
users_schema["id"] = ("Id", "INTEGER PRIMARY KEY")
users_schema["reputation"] = ("Reputation", "INTEGER NOT NULL")
users_schema["creation_date"] = ("CreationDate", "TEXT")
users_schema["display_name"] = ("DisplayName", "TEXT")
users_schema["url"] = ("WebsiteUrl", "TEXT")
users_schema["location"] = ("Location", "TEXT")
users_schema["about_me"] = ("AboutMe", "TEXT")
users_schema["views"] = ("Views", "INTEGER")
users_schema["profile_image"] = ("ProfileImageUrl", "TEXT")
users_schema["account_id"] = ("AccountId", "INTEGER")
users_schema["up_votes"] = ("UpVotes", "INTEGER")


def create_table_if_not_exists(db, name, schema, foreign_keys=[]):
    columns = [f"{col} {typ}" for col, (_, typ) in schema.items()]
    sql = f"""CREATE TABLE IF NOT EXISTS {name} ({", ".join(columns + foreign_keys)});"""
    db.execute(sql)
    
def get_insert_sql(table_name, schema):
    return f"""INSERT INTO {table_name} ({",".join(schema.keys())}) VALUES ({",".join(["?" for _ in range(len(schema))])});"""

def parse_row(schema, root):
    vals = OrderedDict()
    for col, (attr, typ) in schema.items():
        if attr in root.attrib:
            v = root.attrib[attr]
            # Convert to int
            if typ.startswith("INTEGER"):
                v = int(v)
            vals[col] = v
        else:
            vals[col] = None
    return vals


def process_posts(fd, db):
    it = iter(fd)
    # Skip first two lines (xml opening tags)
    for i in range(2):
        next(it)
        
    def pull_rows():
        for line in tqdm(it):
            line = line.strip()
            if line == "</posts>":
                break

            root = ET.fromstring(line)
            row = parse_row(posts_schema, root)

            if row["post_type"] not in (1, 2):
                continue
            
            try:
                yield list(row.values())
            except:
                print(row)

    insert_sql = get_insert_sql("posts", posts_schema)
    
    cur = db.cursor()
    cur.executemany(insert_sql, pull_rows())
    db.commit()

    
def process_users(fd, db):
    it = iter(fd)
    # Skip first two lines (xml opening tags)
    for i in range(2):
        next(it)
            
    def pull_rows():
        for line in tqdm(it):
            line = line.strip()
            if line == "</users>":
                break

            root = ET.fromstring(line)
            row = parse_row(users_schema, root)

            yield list(row.values())
            
    insert_sql = get_insert_sql("users", users_schema)
    
    cur = db.cursor()
    cur.executemany(insert_sql, pull_rows())
    db.commit()

def main(source_root, out_path):
    try:
        db = sqlite3.connect(out_path)
    except Error as e:
        print("An error occured while connecting to database")
        print(e)
        return
    
    db.execute("PRAGMA foreign_keys = ON;")
    
    create_table_if_not_exists(db, "users", users_schema)
    create_table_if_not_exists(db, "posts", posts_schema, ["FOREIGN KEY (user_id) REFERENCES users(id)"])

    with open(os.path.join(source_root, "Users.xml"), "r") as fd:
        process_users(fd, db)

    with open(os.path.join(source_root, "Posts.xml"), "r") as fd:
        process_posts(fd, db)

    db.close()

In [4]:
main(source_root, out_path)

21833it [00:00, 35833.67it/s]
11475it [00:00, 19211.80it/s]


In [5]:
import requests
import re

base_url = """https://archive.org/download/stackexchange"""
response = requests.get(base_url)
body = response.text
matches = re.findall("""td><a\s+href="([^\"]+)"\s*>[^<]+<\/\s*a\s*>\s\(<\s*a\s+href="([^\"]+)"\s*>View Contents<\/\s*a\s*>\)<\/\s*td\s*>""", body)
urls = [m[0] for m in matches]

In [6]:
urls = [url for url in urls if ".meta." not in url]

In [7]:
from datetime import datetime

now = datetime.now()

# dd/mm/YY
script = "\n".join([f"wget -O {url} {base_url}/{url}" for url in urls])
script = f"""
#!/bin/bash

# Generated on {now.strftime("%d/%m/%Y %H:%M:%S")}

{script}
"""

f = open("download.sh", "w")
f.write(script)
f.close()